In [1]:
import pandas as pd
import altair as alt

In [2]:
from datetime import datetime, timedelta
 
def descobre_mes(ano, num_semana):    
    date = str(ano) + '-W' + str(num_semana)
    res = datetime.strptime(date + '-1', "%Y-W%W-%w")
    return res.strftime("%B")

In [3]:
def extrair_elementos(lista_str):
    try:
        lista = ast.literal_eval(lista_str)
        return ', '.join(lista)
    except (SyntaxError, ValueError):
        return None

In [4]:
def remover_duplicatas(lista_str):
    try:
        lista = list(dict.fromkeys(lista_str))
        return lista
    except (SyntaxError, ValueError):
        return None

In [5]:
import ast

hit_songs = pd.read_csv('../songs_data/hit_songs/spotify_hits_dataset_complete.csv', delimiter='\t')
hit_songs = hit_songs.explode(['artist_id','artist_name']).reset_index(drop=True)
hit_songs['artist_id'] = hit_songs['artist_id'].apply(extrair_elementos)

artists = pd.read_csv('../songs_data/artists/spotify_artists_info_complete_reduced_genres.csv', delimiter='\t')
total_charts = pd.read_csv('../my_data/charts/total_charts.csv')

songs_and_artists = pd.merge(hit_songs, artists, on='artist_id')
charts = pd.merge(total_charts, songs_and_artists, on='song_id')


In [6]:
# ajustando japao
charts.loc[(charts['country'] == 'japan') & (charts['year'] == 2017), 'week'] += 34

In [7]:
charts[(charts['country'] == 'japan') & (charts['year']==2017)]['week'].describe()

count    2453.000000
mean       43.642886
std         5.183625
min        35.000000
25%        39.000000
50%        44.000000
75%        48.000000
max        52.000000
Name: week, dtype: float64

In [8]:
charts['month'] = charts.apply(lambda row: descobre_mes(row['year'], row['week']), axis=1)

In [9]:
charts.columns

Index(['position', 'song_id', 'song_name_x', 'artist', 'streams',
       'last_week_position', 'weeks_on_chart', 'peak_position',
       'position_status', 'year', 'week', 'country', 'song_name_y',
       'artist_id', 'artist_name', 'popularity_x', 'explicit', 'song_type',
       'track_number', 'num_artists', 'num_available_markets', 'release_date',
       'duration_ms', 'key', 'mode', 'time_signature', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'valence', 'tempo', 'name', 'followers', 'popularity_y',
       'genres', 'image_url', 'month'],
      dtype='object')

In [10]:
colunas_de_interesse = ['artist_id', 'artist_name', 'year', 'month', 'streams', 'song_name_x', 'genres', 'country', 'image_url']
data = charts[colunas_de_interesse]

In [11]:
agg_func = {
    'artist_name': 'first', 
    'streams': 'sum', 
    'song_name_x': list, 
    'genres': list,  
    'image_url': 'first'
}
data_analysis = data.groupby(['artist_id', 'year', 'month', 'country']).agg(agg_func).reset_index()
data_analysis['genres'] = data_analysis['genres'].apply(lambda x: list(pd.Series(x).explode().unique()))
data_analysis['artist_name'] = data_analysis['artist_name'].apply(extrair_elementos)
data_analysis['song_name_x'] = data_analysis['song_name_x'].apply(remover_duplicatas)
data_analysis.head()

,artist_id,year,month,country,artist_name,streams,song_name_x,genres,image_url
0,00FQb4jTyendYWaN8pK0wa,2017,April,united_kingdom,Lana Del Rey,1147562,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
1,00FQb4jTyendYWaN8pK0wa,2017,February,australia,Lana Del Rey,356843,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
2,00FQb4jTyendYWaN8pK0wa,2017,February,brazil,Lana Del Rey,1108298,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
3,00FQb4jTyendYWaN8pK0wa,2017,February,canada,Lana Del Rey,546300,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...
4,00FQb4jTyendYWaN8pK0wa,2017,February,france,Lana Del Rey,160031,[Love],[['pop']],https://i.scdn.co/image/6e8dc460cfb6b89d797030...


In [12]:
agg_func2 = {
    'streams': 'sum', 
    'song_name_x': 'first', 
    'genres': 'first',  
    'image_url': 'first'
}

top_5_artistas = (
    data_analysis
    .groupby(['artist_id', 'artist_name', 'year', 'month', 'country'])
    .apply(lambda x: x.nlargest(5, 'streams'))
    .reset_index(drop=True)
)

In [13]:
top_5_artistas.info()
data = top_5_artistas.head(1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26969 entries, 0 to 26968
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_id    26969 non-null  object
 1   year         26969 non-null  int64 
 2   month        26969 non-null  object
 3   country      26969 non-null  object
 4   artist_name  26969 non-null  object
 5   streams      26969 non-null  int64 
 6   song_name_x  26969 non-null  object
 7   genres       26969 non-null  object
 8   image_url    26888 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.9+ MB


<h1>Gráfico</h1>

tentando fazer tooltip

In [15]:
# tentando colocar imagens
data2 = data.copy()
data2 = data2.rename(columns={'image_url':'image'})
data2['image'] = data2['image'].astype(str)
data2['image'].dtype

dtype('O')

In [16]:

dropdown_year = alt.binding_select(
    options= data2['year'].unique().tolist(),
    name='Escolha o ano '
)
# year_param = alt.param(
#     value=2017,
#     bind=dropdown_year
# )

dropdown_country = alt.binding_select(
    options= data2['country'].unique().tolist(),
    name='Escolha um país ou visão global '
)
# country_param = alt.param(
#     value='global',
#     bind=dropdown_country
# )

year_select = alt.selection_point(fields=['year'], bind=dropdown_year)
country_select = alt.selection_point(fields=['country'], bind=dropdown_country)

grafico = alt.Chart(data2, width=900, height=500).mark_circle(
    width=50,
    height=50
).encode(
    alt.X('artist_name', type='nominal', title='aaa'),
    alt.Y('streams', type='quantitative', title='Streams'),
    tooltip=['image']
    # tooltip=alt.Tooltip(['image:N'], scale=alt.Scale(domain=[0, 1], range=[0, 1]))
).add_selection(
    year_select,
    country_select
).transform_filter(
    year_select & country_select
).properties(title='Gráfico Filtrado')


filter_year = grafico.add_params(
    year_select
).transform_filter(
    year_select
).properties(title='filtrin')

filter_country = grafico.add_params(
    country_select
).transform_filter(
    country_select
).properties(title='filtrin2')

grafico

c:\Users\larys\OneDrive\Documentos\Sistemas de Informação\2023.2\Visualização de Dados\Trabalhos\Trabalho 02\venv\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)